In [1]:
!pip list

Package                            Version
---------------------------------- -------------------
absl-py                            1.0.0
aiobotocore                        1.2.1
aiohttp                            3.8.1
aioitertools                       0.7.1
aiosignal                          1.2.0
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-project                   0.9.1
anyio                              2.1.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.9.3
astropy                            4.1
astunparse                         1.6.3
async-generator                    1.10
async-timeout                      4.0.2
asynctest                          0.13.0
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.

In [5]:
!pip install mlflow pymysql boto3 mysql

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 88 kB 10.2 MB/s            
  Preparing metadata (setup.py) ... error
  ERROR: Command errored out with exit status 1:
   command: /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/bin/python -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-mc68gd4b/mysqlclient_264823ae8fa345a194509b7be7b7f732/setup.py'"'"'; __file__='"'"'/tmp/pip-install-mc68gd4b/mysqlclient_264823ae8fa345a194509b7be7b7f732/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-e76qi_9j
       cwd: /tmp/pip-install-mc68gd4b/mysqlclient_264823ae8fa345a194509b7be7b7f732/
  Complete output (15 lines):
  /bi

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import mlflow
import mlflow.sklearn

import pymysql

# Connect to the database
connection = pymysql.connect(host='mlflow-backend.chf6ry9cdkyl.eu-central-1.rds.amazonaws.com',
                             user='admin',
                             password='mlflowadmin',
                             database='mlflow-backend',
                             charset='utf8mb4',
                             port=3306,
                             cursorclass=pymysql.cursors.DictCursor,
                             connect_timeout=5)


X = np.linspace(0, 1, 11)
y = X*2
X = np.array([[x] for x in X])
y = np.array([[i] for i in y])

s3_bucket = "s3://mi4people-soil-project/mlflow-artifacts/"
mlflow.create_experiment("test4", s3_bucket)
mlflow.set_experiment('test4')
mlflow.sklearn.autolog()
mlflow.set_tracking_uri("s3://mi4people-soil-project/mlflow-artifacts/")
mlflow.set_registry_uri("s3://mi4people-soil-project/mlflow-backend/")
with mlflow.start_run():
    model = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = sum([np.sqrt((i-j)**2) for i, j in  zip(y_pred, y_test)])
    #mlflow.log_metric("mse", mse)
    #mlflow.log_artifact(model)
# TODO parse mlruns and write to s3? Or set up tracking server?

OperationalError: (2003, "Can't connect to MySQL server on 'mlflow-backend.chf6ry9cdkyl.eu-central-1.rds.amazonaws.com' (timed out)")

In [7]:
!conda install mysql

Solving environment: | ^C
failed with initial frozen solve. Retrying with flexible solve.


In [25]:
!pwd

/home/ec2-user/SageMaker/MI4People_soil_quality/notebooks/utils


In [ ]:
import pymysql
import mysql.connector
import sys
import boto3
import os

ENDPOINT='mlflow-backend.chf6ry9cdkyl.eu-central-1.rds.amazonaws.com'
PORT="3306"
USER="admin"
REGION="eu-central-1c"
DBNAME="mlflow-backend"
os.environ['LIBMYSQL_ENABLE_CLEARTEXT_PLUGIN'] = '1'

#gets the credentials from .aws/credentials
session = boto3.Session(profile_name='default')
client = session.client('rds')

token = client.generate_db_auth_token(DBHostname=ENDPOINT, Port=PORT, DBUsername=USER, Region=REGION)

try:
    conn =  mysql.connector.connect(host=ENDPOINT, user=USER, passwd=token, port=PORT, database=DBNAME, ssl_ca='SSLCERTIFICATE')
    cur = conn.cursor()
    cur.execute("""SELECT now()""")
    query_results = cur.fetchall()
    print(query_results)
except Exception as e:
    print("Database connection failed due to {}".format(e))          
                